# Notification

> This notebook implement notification workflow

In [ ]:
#| default_exp notification

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from threading import Thread, Event
import time
from typing import Optional, Callable
from llmcam.fn_to_fc import complete, tool_schema

In [ ]:
#| export
# Define the stream thread class
class StreamThread(Thread):
    def __init__(self, thread_id, tools, messages):
        super().__init__()
        self.thread_id = thread_id
        self.stop_event = Event()
        self.tools = tools
        self.messages = messages

    def run(self):
        while not self.stop_event.is_set():
            complete(self.messages, tools=self.tools)
            time.sleep(5)

    def stop(self):
        self.stop_event.set()

In [ ]:
#| export
def notification_stream_core(
    tools: list,  # Tools to use
    messages: list,  # Previous conversation with the user
    stream_starter: Optional[Callable] = None,  # Function to start the stream
    send_notification: Optional[Callable] = None,  # Function to send the notification
    stream_stopper: Optional[Callable] = None,  # Function to stop the stream
    send_notification_schema: Optional[dict] = None,  # Schema for the send_notification function
    stream_stopper_schema: Optional[dict] = None,  # Schema for the stream_stopper function
) -> str:
    """Core function to start and stop the notifications stream"""
    # Copy the messages to avoid modifying the original list
    submessages = [ message for message in messages ]

    # Extract subtools schemas
    send_notification_schema = send_notification_schema or tool_schema(send_notification, 'send_notification')
    stream_stopper_schema = stream_stopper_schema or tool_schema(stream_stopper, 'send_notification')

    # Add sending notification services to tool schema
    subtools = [ tool for tool in tools if tool['function']['name'] != 'start_notification_stream' ]
    subtools.append(send_notification_schema)
    subtools.append(stream_stopper_schema)

    # Start the notifications stream
    stream_starter(subtools, submessages)

    return 'Notifications stream started'

In [ ]:
#| export
def default_stream_starter(tools, messages):
    """Default function to start the notifications stream"""
    global stream_thread

    # Start the notifications stream
    stream_thread = StreamThread(1, tools, messages)
    stream_thread.start()

def default_stream_stopper():
    """Default function to stop the notifications stream"""
    global stream_thread

    # Stop the notifications stream
    stream_thread.stop()
    stream_thread.join()

In [ ]:
#| export
def process_notification_schema(
    start_notifications_stream: Callable,  # Function to start the notifications stream
):
    """Process the notification schema"""
    notification_schema = tool_schema(start_notifications_stream, 'notification')

    notification_schema['function']['parameters'] = {
        'type': 'object',
        'properties': {
            'messages': {
                'description': 'All the previous messages in the conversation',
                'type': 'array',
                'items': {
                    'type': 'object',
                    'properties': {
                        'role': {
                            'type': 'string',
                            'enum': ['user', 'tool', 'system', 'assistant']
                        },
                        'content': {
                            'type': 'string'
                        }
                    }
                }
            }
        }
    }
    return notification_schema

Simulated workflow:

In [ ]:
from llmcam.fn_to_fc import complete, form_msgs, form_msg, tool_schema, print_msgs
import random

In [ ]:
tools = []

In [ ]:
def random_generator():
    """Generate a random number between 1 and 100"""
    return random.randint(1, 100)

In [ ]:
notifications = []
def send_notification(msg: str):
    """Send a notification"""
    notifications.append(msg)
    return notifications

In [ ]:
def start_notification_stream(
    messages: list  # Previous conversation with the user
):
    return notification_stream_core(
        tools, 
        messages,
        stream_starter=default_stream_starter,
        send_notification=send_notification,
        stream_stopper=default_stream_stopper
    )

In [ ]:
tools.append(tool_schema(random_generator, 'random_generator'))
tools.append(process_notification_schema(start_notification_stream))

In [ ]:
tools

[{'type': 'function',
  'function': {'name': 'random_generator',
   'description': 'Generate a random number between 1 and 100',
   'parameters': {'type': 'object', 'properties': {}, 'required': []},
   'metadata': {'module': '__main__', 'service': 'random_generator'}}},
 {'type': 'function',
  'function': {'name': 'start_notification_stream',
   'description': 'No description provided.',
   'parameters': {'type': 'object',
    'properties': {'messages': {'description': 'All the previous messages in the conversation',
      'type': 'array',
      'items': {'type': 'object',
       'properties': {'role': {'type': 'string',
         'enum': ['user', 'tool', 'system', 'assistant']},
        'content': {'type': 'string'}}}}}},
   'metadata': {'module': '__main__', 'service': 'notification'}}}]

In [ ]:
#| eval: false
messages = form_msgs([
    ('system', 'You are a helpful system administrator. Use the supplied tools to assist the user. \
If asked to monitor and notify, do not resolve in main thread but defer to sub GPT in notifcation stream instead.'),
    ('user', 'Can you notify me every time you generate a number higher than 50? Stop after 10 notifications.'),
])
complete(messages, tools=tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to assist the user. If asked to
monitor and notify, do not resolve in main thread but defer to sub GPT in notifcation stream
instead.
>> User:
Can you notify me every time you generate a number higher than 50? Stop after 10 notifications.
>> Assistant:
I've started a notification stream to monitor the generation of numbers higher than 50. You will
receive notifications when such numbers are generated, up to 10 times.


In [ ]:
#| eval: false
messages

[{'role': 'system',
  'content': 'You are a helpful system administrator. Use the supplied tools to assist the user. If asked to monitor and notify, do not resolve in main thread but defer to sub GPT in notifcation stream instead.'},
 {'role': 'user',
  'content': 'Can you notify me every time you generate a number higher than 50? Stop after 10 notifications.'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_XyyPi4ASAwZhUYCOG0HalFy8',
    'function': {'arguments': '{"messages":[{"role":"user","content":"Can you notify me every time you generate a number higher than 50? Stop after 10 notifications."}]}',
     'name': 'start_notification_stream'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': '{"messages": [{"role": "user", "content": "Can you notify me every time you generate a number higher than 50? Stop after 10 notifications."}], "start_notification_stream": "Notifications stream started"}',
  'tool_call_id': 'call_XyyPi4ASAwZhU

In [ ]:
#| eval: false
for noti in notifications:
    print(noti)

Generated number 70 is higher than 50.
Generated number 62 is higher than 50.
Generated number 96 is higher than 50.
Generated number 97 is higher than 50.
Generated number 77 is higher than 50.
Generated number 100 is higher than 50.
Generated number 93 is higher than 50.
Generated number 76 is higher than 50.
Generated number 57 is higher than 50.
Generated number 57 is higher than 50.


In [ ]:
#| eval: false
len(notifications)

10

In [ ]:
#| eval: false
stream_thread.is_alive()

False

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()